# **Genetik Algoritma ile Griewank Optimizasyonu**

### 1. Griewank Fonksiyonu

Griewank fonksiyonu şu şekilde tanımlanır:

![Görsel açıklaması](https://mathworld.wolfram.com/images/equations/GriewankFunction/NumberedEquation1.svg)

In [1]:
import numpy as np

def griewank(x):
    sum_term = np.sum(x**2) / 4000
    prod_term = np.prod(np.cos(x / np.sqrt(np.arange(1, len(x) + 1))))
    return sum_term - prod_term + 1

# Örnek kullanım
x = np.array([0, 0, 0, 0])  # Global minimum noktası
print(griewank(x))  # Sonuç: 0

x = np.array([1, 1, 1, 1])
print(griewank(x))  # Başka bir örnek nokta

0.0
0.6989516489586614


### 2. Genetik Algoritma

#### 2.1 Popülasyon

Popülasyon, bireylerden (veya kromozomlardan) oluşur ve her birey potansiyel bir çözüm temsil eder. Popülasyonun temel işlevi, çeşitli çözümler arasından en iyi olanı bulmaya yönelik evrimsel süreçlerin (seçim, çaprazlama, mutasyon) uygulanmasını sağlamaktır. Genetik algoritmanın başarısı için kritik öneme sahiptir. İyi bir başlangıç popülasyonu, çözüm uzayında daha etkili bir arama yapılmasına olanak tanır. Ayrıca, yeterli çeşitliliğe sahip bir popülasyon, yerel minimumlara takılmadan global optimum çözüme ulaşma şansını artırır.

In [2]:
import random

class RandomPopulation():
    def __init__(self, lower_band, upper_band, num_variables, population_size):
        self.lower_band = lower_band
        self.upper_band = upper_band
        self.num_variables = num_variables
        self.population_size = population_size

    def create(self):
        #population = [[random.uniform(self.lower_band, self.upper_band) for _ in range(self.num_variables)] for _ in range(self.population_size)]
        population = np.random.uniform(self.lower_band, self.upper_band, (self.population_size, self.num_variables))
        return population


In [3]:
population = RandomPopulation(-600, 600, 30, 50)

#### 2.1 Seçim 

Seçim, genetik algoritmanın evrimsel sürecinde kritik bir adımdır. Popülasyon içindeki bireylerin, onların fitness değerlerine dayanarak seçilmesini içerir. Yüksek fitness değerine sahip bireylerin seçilme olasılığı daha yüksektir, bu da genetik çeşitliliği ve popülasyonun genel kalitesini artırmaya yardımcı olur.

In [4]:
class Selection:
    def __init__(self, population_size):
        self.population_size = population_size

    def get_population_size(self):
        return self.population_size

    def choose(self):
        pass



#### 2.1.1. Rastgele Seçimi
Rastgele seçme, popülasyondaki bireylerin herhangi bir fitness değeri dikkate alınmaksızın tamamen rastgele seçilmesi yöntemidir. Bu yöntem, her bireyin seçilme olasılığının eşit olduğu anlamına gelir.

In [5]:
class RandomSelection(Selection):
    def __init__(self, population_size):
        super().__init__(population_size)

    def choose(self):
        numbers = np.array([0, 0])

        numbers[0] = random.randint(0, self.get_population_size()-1)
        numbers[1] = numbers[0]

        while numbers[0] == numbers[1]:
            numbers[1] = random.randint(0, self.get_population_size()-1)

        return numbers

#### 2.1.2. Rulet Tekerleği Seçimi

Bu yöntem, her bireyin fitness değerine orantılı olarak seçilme olasılığına sahip olduğu bir şans oyununa benzer.
Yüksek fitness değerine sahip bireylerin seçilme olasılığı daha yüksektir.

In [22]:
class RouletteSelection(Selection):
    def __init__(self, population_size):
        super().__init__(population_size)

    def choose(self, populasyonF):
        fitness_oranlari = [0.0] * self.get_population_size()
        top_fitness = 0
        for sayi in populasyonF:
            top_fitness += sayi

        i = 0
        for sayi in populasyonF:
            fitness_oranlari[i] = (sayi / top_fitness) * 100
            i += 1

        numbers = [0, 0]

        numbers[0] = random.randint(0, 100)
        numbers[1] = numbers[0]

        while numbers[0] == numbers[1]:
            numbers[1] = random.randint(0, self.get_population_size()-1)

        for j in range(len(numbers)):
            dilim = 0
            for k in range(len(fitness_oranlari)):
                dilim += fitness_oranlari[k]
                if dilim >= numbers[j]:
                    numbers[j] = k
                    break

        return numbers

#### 2.1.3. Turnuva Seçimi 

Rastgele seçilen birkaç birey (genellikle 2 veya daha fazla) arasında turnuva düzenlenir ve en yüksek fitness değerine sahip olan birey seçilir.
Turnuva büyüklüğü, seçilen bireylerin sayısını ifade eder ve daha büyük turnuva büyüklükleri, daha iyi bireylerin seçilme olasılığını artırır.

In [7]:
class TournamentSelection(Selection):
    def __init__(self, population_size):
        super().__init__(population_size)

    def choose(self, fitness):
        numbers = [0, 0]
        numbers2 = [0, 0]

        for j in range(2):
            numbers[0] = random.randint(0, self.get_population_size()-1)
            numbers[1] = numbers[0]

            while numbers[0] == numbers[1]:
                numbers[1] = random.randint(0, self.get_population_size()-1)
            if fitness[numbers[1]] > fitness[numbers[0]]:
                numbers2[j] = numbers[1]
            else:
                numbers2[j] = numbers[0]

        return numbers2

#### 2.2. Çaprazlama (Crossover)
İki ebeveyn bireyin genetik bilgilerini birleştirerek yeni bireyler (çocuklar) oluşturma sürecidir.

In [8]:
import random

class CrossOver:
    def __init__(self):
        self.random = random.Random()

    def get_random(self):
        return self.random

    def make(self, x1, x2):
        pass

class SingleCrossOver(CrossOver):
    def make(self, x1, x2):
        cons = np.array([None, None])
        cons[0] = [0] * len(x1)
        cons[1] = [0] * len(x1)

        index = self.get_random().randint(1, len(x1))

        for i in range(index):
            cons[0][i] = x1[i]
            cons[1][i] = x2[i]

        for i in range(index, len(x1)):
            cons[1][i] = x1[i]
            cons[0][i] = x2[i]

        return cons

#### 2.3. Mutasyon
Bireylerin genetik yapılarında rastgele küçük değişiklikler yaparak çeşitliliği artırma sürecidir.

In [9]:
import random
import math

class Mutation:
    def __init__(self, constant):
        self.random = random.Random()
        self.constant = constant

    def get_random(self):
        return self.random

    def get_constant(self):
        return self.constant

    def make(self, x):
        pass

class SingleMutation(Mutation):
    def __init__(self, constant):
        super().__init__(constant)

    def make(self, x):
        index = self.get_random().randint(0, len(x) - 1)
        pow = self.get_random().randint(0, 1)
        x[index] = x[index] + math.pow(-1, pow) * self.get_constant()

        return x

#### 2.4 Limit (Ceza Fonksiyonları)
Cezalandırma fonksiyonları, genetik algoritmalarda kısıtlamaları ihlal eden çözümleri cezalandırmak için kullanılan yöntemlerdir. Bu fonksiyonlar, kısıtlamalara uymayan çözümler için fitness değerlerini düşürerek, bu tür çözümlerin seçilme olasılığını azaltır. Cezalandırma fonksiyonları, genetik algoritmanın kısıtlanmış optimizasyon problemlerini çözebilmesini sağlar.

In [10]:
class Bound:
    def __init__(self, lower_band, upper_band):
        self.lower_band = lower_band
        self.upper_band = upper_band
    
    def get_lower_band(self):
        return self.lower_band

    def get_upper_band(self):
        return self.upper_band

    def make(self, x):
        pass


#### 2.4.1. Sınırlara Eşitleme
Değer sınırları aşarsa sınıra eşitlenir.

In [11]:
class LimitBound(Bound):
    def __init__(self, lower_band, upper_band):
        super().__init__(lower_band, upper_band)
    
    def make(self, x):
        for i in range(len(x)):
            if x[i] < self.get_lower_band():
                x[i] = self.get_lower_band()
            elif x[i] > self.get_upper_band():
                x[i] = self.get_upper_band()
        return x



#### 2.4.2. Rasgele Değer Üretme
Değer sınırları aşarsa rasgele yeni bir değer üretilir.

In [12]:
class RandomBound(Bound):
    def __init__(self, lower_band, upper_band):
        super().__init__(lower_band, upper_band)

    def make(self, x):
        import random
        for i in range(len(x)):
            if x[i] < self.get_lower_band() or x[i] > self.get_upper_band():
                x[i] = random.random() * (self.get_upper_band() - self.get_lower_band()) + self.get_lower_band()
        return x

### 3. Genetik Algoritma ile Griewark Optimizasyon Uygulaması

In [25]:
mutation_constant = 0.1  # mutasyon sabiti
population_size = 50  # popülasyon büyüklüğü
max_iteration = 500000  # max iterasyon sayısı

lower_band = -600 # griewark için alt sınır
upper_band = 600 # griewark için üst sınır

parameter_count = 30 # fonksiyon girdi parametre sayısı 

random_population = RandomPopulation(lower_band, upper_band, parameter_count, population_size)

selection_mutation = RandomSelection(population_size)
#selection = RouletteSelection(population_size)
selection = TournamentSelection(population_size)
cross_over = SingleCrossOver()
mutation = SingleMutation(mutation_constant)
bound = LimitBound(lower_band, upper_band)

fitness = [0.0] * population_size
index = None
consM = None
consC = None
fitnes = 0.0

population = random_population.create()
population[:1]

array([[ 437.57782701,  394.83154143, -245.70329327,  177.58455285,
         -13.11878932,  537.47867097, -266.0542207 ,  294.26412488,
         176.62762546,   89.81425479, -549.27282921,  154.39236403,
         509.13618816, -214.93761269, -259.04662564, -231.69790602,
         121.84556621,    1.30824792,  485.88244256,  -65.51052316,
        -321.04914921, -337.99453986,   49.03248751, -132.93272979,
         202.08840036, -599.91180909, -302.42785277, -589.18545827,
         374.59471428,  263.95063516]])

In [26]:
for i in range(population_size):
    fitness[i] = griewank(population[i])
print(griewank(population[0]))
print(fitness)
print("\n----------------------")
fe = 0  # iterasyon sayacı
while fe < max_iteration:

    index = selection.choose(fitness)
    #print(str(fe)+". iterasyon seçine index"+str(index))
    index[1] = fitness.index(min(fitness))
    consC = cross_over.make(population[index[0]], population[index[1]])
    #print(consC[0])
    fitnes = griewank(np.array(consC[0]))
    fe += 1
    if fitnes < fitness[index[0]]:
        population[index[0]] = consC[0]
        fitness[index[0]] = fitnes

    fitnes = griewank(np.array(consC[1]))
    fe += 1
    if fitnes < fitness[index[1]]:
        population[index[1]] = consC[1]
        fitness[index[1]] = fitnes

    index = selection_mutation.choose()
    consM = mutation.make(population[index[0]])
    bound.make(consM)

    fitnes = griewank(consM)
    fe += 1
    if fitnes < fitness[index[0]]:
        population[index[0]] = consM
        fitness[index[0]] = fitnes

global_min = min(fitness)
print("Global minimum: "+str(global_min))

805.6623850959696
[805.6623850959696, 734.1141861423442, 823.9006152163084, 751.7297260090114, 1153.4835898837296, 1006.9133445219592, 872.1560429262637, 728.2117590235351, 852.2031567550686, 670.3693987708633, 880.1275659119423, 839.1177956227966, 718.3464007682481, 958.1668278337862, 944.6666093352949, 937.4225770123053, 903.490997416426, 831.0883570724928, 830.0430073196535, 742.550503839077, 1015.7554088095401, 868.0636586948393, 1151.2019049479459, 1266.848374447596, 597.3244068581726, 1176.1299519006943, 986.6278860512069, 617.4128931014135, 865.7229734654499, 783.5669660831429, 705.715018019155, 611.701322854542, 946.2877157854211, 927.4165625692781, 1060.518224265385, 755.3943186643543, 748.2051677272776, 1040.388738045361, 929.2287118108612, 933.7574246177724, 909.9039346536074, 729.1130653629027, 898.0800889434532, 1203.6753348425887, 984.2410397774994, 779.7591910979461, 1050.6338165319446, 678.9699538363104, 759.0267278171423, 809.6021531489654]

----------------------
Glob